In [1]:
pip install tensorflow

In [4]:
pip install tensorflow.keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
file_path = '/content/drive/My Drive/Project/corpus.txt'

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Replace with your actual file path
file_path = '/content/drive/My Drive/Project/corpus.txt'

# Load the corpus
with open(file_path, "r") as file:
    corpus = file.read().splitlines()

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Split into X and y
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the model
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len - 1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile and train
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=300, verbose=1)


Epoch 1/300


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0275 - loss: 5.6129
Epoch 2/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0697 - loss: 5.5489
Epoch 3/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0688 - loss: 5.0837
Epoch 4/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0674 - loss: 4.9608
Epoch 5/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0558 - loss: 5.0644
Epoch 6/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0595 - loss: 5.0429
Epoch 7/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0643 - loss: 5.0318
Epoch 8/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0680 - loss: 4.9328
Epoch 9/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0794 - loss: 4.9624
Epoch 10/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0877 - loss: 4.9834
Epoch 11/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1043 - loss: 4.7436
Epoch 12/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy:

In [6]:
def predict_next_word(model, tokenizer, input_text):
    token_list = tokenizer.texts_to_sequences([input_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted)

    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return ""

# Try predictions on test inputs
test_sentences = [
    "I love reading",
    "She enjoys walking",
    "He likes to play",
    "They are going to",
    "We should prepare",
    "It is a beautiful",
    "The children are",
    "I am learning",
    "She is writing a",
    "They are playing"
]

for sent in test_sentences:
    next_word = predict_next_word(model, tokenizer, sent)
    print(f"{sent} ➜ {next_word}")


I love reading ➜ books
She enjoys walking ➜ in
He likes to play ➜ football
They are going to ➜ the
We should prepare ➜ dinner
It is a beautiful ➜ day
The children are ➜ playing
I am learning ➜ how
She is writing a ➜ novel
They are playing ➜ board
